In [ ]:
!pip uninstall scikit-learn -y  | egrep installed
!pip install scikit-learn ipympl | egrep installed

In [ ]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, FileLink, HTML
import subprocess
import sys
import pickle
import tempfile
import os
import base64
import io

In [ ]:
class DemoData:
    
    def __init__(self):
        self.data = []
        self.models = []
        self.btn_gen = widgets.Button(description="Generate Data")
        self.btn_gen.on_click(self.generate_data)
        self.btn_reset = widgets.Button(description="Reset Data")
        self.btn_reset.on_click(self.reset_data)
        self.btn_download = widgets.Button(description="Download Data")
        self.btn_download.on_click(self.download_data)
        self.text_download = widgets.Label(value="")
        self.btn_inference = widgets.Button(description="Inference")
        self.btn_inference.on_click(self.inference)
        self.text_inference = widgets.Label(value="")
        self.btn_pipeline = widgets.Button(description="Run training pipeline")
        self.btn_pipeline.on_click(self.run_pipeline)
        self.num_data = widgets.IntText(value=1, description="# of data to generate", disabled=False)
        self.text_pipeline = widgets.Label(value="")
        self.slope = 3
        self.intercept = 2
        self.stddev = 100
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot()
        self.ax.set_title("Data")
        self.pipeline = widgets.FileUpload(description="Pipeline Upload", multiple=False)
        self.download_output = widgets.Output()
        display(widgets.HBox(children=(self.btn_gen, self.num_data)))
        display(self.btn_reset)
        display(widgets.HBox(children=(self.btn_download, self.text_download)))
        display(widgets.HBox(children=(self.btn_inference, self.text_inference)))
        display(self.pipeline)
        display(widgets.HBox(children=(self.btn_pipeline, self.text_pipeline)))
        display(self.download_output)
        
    def generate_data(self, btn):
        #self.output.clear_output()
        to_append = [
            {
                "x": len(self.data) + x,
                "y": (len(self.data) + x) * self.slope + self.intercept + self.stddev * np.random.randn(),
            } for x in range(self.num_data.value)
        ]
        self.data += to_append
        self.ax.clear()
        df = pd.DataFrame(self.data)
        df.plot(y="y", x="x", ax=self.ax, marker="x")
        self.fig.canvas.draw()
        
    def reset_data(self, btn):
        self.data = []
        self.ax.clear()
        self.fig.canvas.draw()
        
    def download_data(self, btn):
        if not self.data:
            self.text_download.value = "No Data"
            return
        with io.BytesIO() as buf:
            df = pd.DataFrame(self.data)
            df.to_csv(buf)
            encoded = base64.b64encode(buf.getvalue())
        payload = encoded.decode()
        with self.download_output:
            self.download_output.clear_output()
            display(HTML(f"""
<html>
<body>
<a id="asdf" download="data_dumped" href="data:text/csv;base64,{payload}" download>
</a>

<script>
(function download() {{
document.getElementById('asdf').click();
}})()
</script>

</body>
</html>
"""))
        
    def inference(self, btn):
        if not self.data:
            self.text_inference.value = "No data"
            return
        self._plot_model()
        
    def _plot_model(self):
        if not self.models:
            self.text_inference.value = "No model"
            return
        df = pd.DataFrame(self.data)
        regr = self.models[-1]
        self.text_inference.value = f"slope: {regr.coef_[0]:.2f}, intercept: {regr.intercept_:.2f}"
        df["model"] = regr.predict(df["x"].values[:, None])
        self.ax.clear()
        df.plot(y="y", x="x", ax=self.ax)
        df.plot(y="model", x="x", ax=self.ax)
        self.fig.canvas.draw()
        
    def run_pipeline(self, btn):
        if not self.pipeline.value:
            self.text_pipeline.value = "No pipeline"
            return
        if not self.data:
            self.text_pipeline.value = "No data"
            return
        self.text_pipeline.value = ""
        with tempfile.TemporaryDirectory() as tmpdir:
            pipeline_path = os.path.join(tmpdir, "pipeline.py")
            data_path = os.path.join(tmpdir, "data.csv")
            df = pd.DataFrame(self.data)
            df.to_csv(data_path)
            with open(pipeline_path, "wb") as fp:
                fp.write(next(iter(self.pipeline.value.values()))["content"])
            model_path = os.path.join(tmpdir, "model.pkl")
            subprocess.run(
                [
                    sys.executable,
                    pipeline_path,
                    "--datapath",
                    data_path,
                    "--modelpath",
                    model_path,
                ], check=True,
            )
            with open(model_path, "rb") as fp:
                self.models.append(pickle.load(fp))
        self._plot_model()
        
        
demo = DemoData()